# 파트 03
----
아래에서는 인구이동 데이터의 1차적인 전처리 과정을 행한다. 우선적으로 읍면동 부분을 제외한 행자부코드를 이용하여 하나의 지역코드로 묶는다. 이후 불필요한 칼럼을 없앤다.

2012년과 2013년의 데이터의 경우 이동 총인구가 마침표로 잘못 기입된 데이터가 있어 이를 수정하고 정수타입으로 변환한다.

또한 누락된 데이터 외의 모든 데이터는 정수타입으로 표기될 수 있으므로 nan값을 -1로 수정 한뒤 정수타입으로 변환시킨다.

세대주 성별의 경우 주민등록상 등록된 번호로 구분이 되어 있기에 일괄적으로 대응되는 1 또는 2로 변환한다.

세대주 구분에서 0과 -1은 모두 비세대주로 간주하여 0으로 통일한다.

마지막으로 데이터 탐색 단계에서 이동 총인구가 -1이거나나 0으로 기입된 데이터 포인트가 119개 있다는 것을 알수 있었으며, 이 경우 이동 남인구와 이동 여인구의 데이터가 모두 0으로 되어있는 것을 감안, 그리고 전체 데이터 크기인 약 6300만과 비교하여 상당히 적은 수인 것도 감안하여 이동 총인구가 0보다 작거나 같은 경우 누락시키기로 한다.

위의 전처리 과정을 위한 데이터 탐구 과정(이동 총인구 데이터의 타입, 세대주 성별 구분 코드, 세대주 관계 코드의 비일관성 발견 등)은 아래 코드에는 생략한다.

In [1]:
%matplotlib inline
import glob
from __future__ import print_function
from __future__ import division

In [2]:
path = 'raw_data\\'

In [3]:
population_columns = [u'전입_시도', u'전입_시군구', u'전입_동읍면', u'전입_년', u'전입_월', u'전입_일',
                      u'전출_시도', u'전출_시군구', u'전출_동읍면', u'사유',
                      u'세대주_관계', u'세대주_나이', u'세대주_성별', u'세대_관련',
                      u'이동_총인구', u'이동_남인구', u'이동_여인구']

In [4]:
population_files = glob.glob(path + 'population_*.txt')

In [5]:
# read in data files and output them one by one after scrubbing some data
for population_file in population_files:
    df_temp = pd.read_csv(population_file, names=population_columns)
    
    # create a column with combined address codes
    df_temp[u'전입코드'] = df_temp[u'전입_시도'] * 1000 + df_temp[u'전입_시군구']
    df_temp[u'전출코드'] = df_temp[u'전출_시도'] * 1000 + df_temp[u'전출_시군구']
    
    # drop unnecessary columns
    df_temp.drop([u'전입_동읍면', u'전입_일', u'전출_동읍면',
                  u'전입_시도', u'전입_시군구', u'전출_시도', u'전출_시군구'], axis=1, inplace=True)
    
    # this is to handle exceptions in years 2012 and 2013
    if (population_file[-8:-4] == '2012') | (population_file[-8:-4] == '2013'):
        df_temp.loc[df_temp[u'이동_총인구'] == '.', u'이동_총인구'] = '-1'
        df_temp[u'이동_총인구'] = df_temp[u'이동_총인구'].astype(int)
    
    # we are converting nan values to -1 so that we may use integer type to save space
    df_temp[df_temp.isnull()] = -1
    
    # change gender codes to 1 or 2 accordingly
    df_temp.loc[(df_temp[u'세대주_성별'] == 9) | (df_temp[u'세대주_성별'] == 3), u'세대주_성별'] = 1
    df_temp.loc[(df_temp[u'세대주_성별'] == 0) | (df_temp[u'세대주_성별'] == 4), u'세대주_성별'] = 2
    
    # change missing data values for consistency
    df_temp.loc[df_temp[u'세대주_관계'] == -1, u'세대주_관계'] = 0
    
    # after some exploratory data analysis, we find that there are
    # total of 119 data points with total population moved with <= 1
    # as 119 is a relatively small amount of data, we drop them before exporting to csv
    # also, we convert every data to integer type to save some space
    df_temp.loc[df_temp[u'이동_총인구'] > 0].astype(int).to_csv(
        path + '03_pop_' + population_file[-8:-4] + '.csv',
        index=False,
        encoding='utf-8'
    )
    print(population_file)

raw_data\population_change_2006.txt
raw_data\population_change_2007.txt
raw_data\population_change_2008.txt
raw_data\population_change_2009.txt
raw_data\population_change_2010.txt
raw_data\population_change_2011.txt


C:\ProgramData\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


raw_data\population_change_2012.txt
raw_data\population_change_2013.txt
raw_data\population_change_2014.txt
raw_data\population_change_2015.txt
